# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Load the file
recipes = pd.read_csv("recipes.tsv", sep="\t")

# Inspect data
recipes.head()

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...


In [6]:
recipes.columns.tolist()

['recipe_slug', 'recipe_title', 'recipe_introduction']

In [7]:
len(recipes)

100

In [8]:
# Copy into a new DataFrame
ratings = recipes[["recipe_slug"]].copy()

# Define rules
def assign_rating(title, intro):
    title_lower = title.lower()
    intro_lower = intro.lower()
    
    # dislikes: bacon
    if "bacon" in title_lower or "bacon" in intro_lower:
        return 0.0
    
    # likes: desserts, sweets
    sweet_keywords = ["cake", "cookie", "brownie", "pie", "ice cream", "pudding",
                      "dessert", "crisp", "candy", "sweet", "chocolate", "muffin"]
    if any(word in title_lower for word in sweet_keywords):
        return 1.0
    
    # medium-high likes: fruits
    fruit_keywords = ["apple", "banana", "peach", "pear", "berry", "lemon"]
    if any(word in title_lower for word in fruit_keywords):
        return 0.75
    
    # medium-low likes: spam, meats
    if "spam" in title_lower or "meat" in title_lower:
        return 0.25
    
    # default: middle ground
    return 0.5

# Apply the rules
ratings["rating"] = recipes.apply(lambda row: assign_rating(row["recipe_title"], row["recipe_introduction"]), axis=1)

# Ensure at least 10 ratings of 0 and 1
n_zeros = (ratings["rating"] == 0).sum()
n_ones = (ratings["rating"] == 1).sum()

# If not enough 0's, force a few extra
if n_zeros < 10:
    extra_idx = ratings[ratings["rating"] == 0.5].sample(10 - n_zeros, random_state=42).index
    ratings.loc[extra_idx, "rating"] = 0.0

# If not enough 1's, force a few extra
if n_ones < 10:
    extra_idx = ratings[ratings["rating"] == 0.5].sample(10 - n_ones, random_state=42).index
    ratings.loc[extra_idx, "rating"] = 1.0

# Save 
output_path = "ratings.tsv"
ratings.to_csv(output_path, sep="\t", index=False)

# Summary counts of ratings
ratings["rating"].value_counts()


rating
0.50    59
1.00    18
0.00    15
0.75     6
0.25     2
Name: count, dtype: int64

In [9]:
ratings.head()

,recipe_slug,rating
0,falafel,0.50
1,spamburger,0.25
2,bacon-fried-rice,0.00
3,chicken-fingers,0.50
4,apple-crisp,1.00


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [ ]:
# YOUR CHANGES HERE

...

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [ ]:
# YOUR CHANGES HERE

...

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [ ]:
# YOUR CHANGES HERE

...

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

...

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [ ]:
# YOUR CHANGES HERE

...

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.